### End to end pipeline
- Integrate the image classification module and the text recipe data module to provide recommendations based on the identified pictures of veggies.
- Load our trained model
- Load all the data
- Classify the images and use matching algorithm to suggest recommendations of recipes
- Types of recommendation:
    - 1) Recipes having all three veggies
    - 2) Recipes having only two of three veggies
    - 3) Recipes having only one of three veggies

In [1]:
import os
from kerasUtil import *
from recipeScript import *
from miscScript import *

Using TensorFlow backend.
c:\users\heman\.conda\envs\tensor\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\heman\Documents\DS\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Load recipe data

In [2]:
#de-serialize the dataframe from step 3 
with open('df_epi_ingred.pkl','rb') as fin:
    df = pickle.load(fin)

In [3]:
df.columns

Index(['id', 'title', 'ingredients', 'tags', 'picture_link', 'instructions',
       'url'],
      dtype='object')

### Integration

In [4]:
extractor = callModel('monet') #kerasUtil.py
predictor = models.load_model('models/trained_model.hdf5')  #load_model is from keras
class_dictionary = np.load('models/train_class_indices.npy').item()
inv_map = {v:k for k,v in class_dictionary.items()}

In [5]:
# Specify directory of test data (images)
image_path_dir = 'data/test/'

print('... models loaded!')

# Loop over each file
result=[]
for pic_file in os.listdir(image_path_dir):
    
    image = load_img(image_path_dir+pic_file, target_size=(224,224))
    image = img_to_array(image) / 255
    image = np.expand_dims(image, axis=0)
    
    # extractor is called above 
    features = extractor.predict(image)
    # predictor
    class_predicted = predictor.predict_classes(features)
    probabilities = predictor.predict_proba(features)
    
    inID = class_predicted[0]
    label = inv_map[inID]
    
    result.append(label)
    
print('The images are: ', ', '.join(result))

# Find recipes with intersections
d3, d2, d1, d = intersect(df, result)
# Output recipes
out3, link3 =outputRecipes(d3)
out2, link2 =outputRecipes(d2)
out1, link1 =outputRecipes(d1)

... models loaded!
The images are:  kale, broccoli, tomato


100%|██████████████████████████████████████████████████████████████████████████| 35965/35965 [00:07<00:00, 4791.45it/s]


1. Recipes with kale: 351, broccoli: 384, tomato: 4704
2. Recipes using all 3 ingredients:  3
3. Recipes using 2 of 3 ingredients:  115
4. Recipes using 1 of 3 ingredients:  4704
5. Total recipe collection:  4717


In [6]:
print('Here are the top recipes for all 3 ingredients: ','\n')
for h,l in zip(out3.title.head(3),link3):
    print(h, end=" ")
    print(l,'\n')
    
print('Here are the top recipes for 2 of 3 ingredients: ','\n','\n')
for h,l in zip(out2.title.head(3),link2):
    print(h, end=" ")
    print(l,'\n')

print('Here are the top recipes for (at least) 1 of 3 ingredients: ','\n','\n')
for h,l in zip(out1.title.head(3),link1):
    print(h, end=" ")
    print(l,'\n')

Here are the top recipes for all 3 ingredients:  

Skillet Greens with Cumin and Tomatoes http://www.epicurious.com/recipes/food/views/skillet-greens-with-cumin-and-tomatoes-241866 

Fresh Garden Wrap http://www.epicurious.com/recipes/food/views/fresh-garden-wrap 

Quick Minestrone Soup http://www.epicurious.com/recipes/food/views/quick-minestrone-soup-236017 

Here are the top recipes for 2 of 3 ingredients:  
 

Presto Pizza http://www.epicurious.com/recipes/food/views/presto-pizza-230141 

Rice with Kale and Tomatoes http://www.epicurious.com/recipes/food/views/rice-with-kale-and-tomatoes-13399 

Broccoli and Cherry Tomato Salad http://www.epicurious.com/recipes/food/views/broccoli-and-cherry-tomato-salad-2218 

Here are the top recipes for (at least) 1 of 3 ingredients:  
 

Crushed Tomatoes http://www.epicurious.com/recipes/food/views/crushed-tomatoes-51255670 

Crunchy Oil-Cured Tomatoes http://www.epicurious.com/recipes/food/views/crunchy-oil-cured-tomatoes-51108070 

Tomatoes w